## Mount with your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup darknet environment

In [ ]:
import os
assert os.getcwd()=='/content', 'Directory should be "/content" instead of "{}"'.format(os.getcwd())

# remove the existing folder if have
!rm -r darknet_for_colab

# download and compile darknet_for_colab
!git clone https://github.com/CZ3004-Group-13/darknet_for_colab.git
%cd darknet_for_colab
!make
!chmod +x ./darknet

## Download yolov4 pre-trained weights

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())
!cp /content/drive/MyDrive/YOLOv4_weight/backup/yolov4_custom_train_last.weights /content/darknet_for_colab

## Download custom dataset (yolo format)

In [ ]:
%cd data
assert os.getcwd()=='/content/darknet_for_colab/data', 'Directory should be "/content/darknet_for_colab/data" instead of "{}"'.format(os.getcwd())

# download data set from google drive
!cp /content/drive/MyDrive/ts.zip /content/darknet_for_colab/data
!unzip ts.zip
!rm -f ts.zip
!ls
%cd ..

## Visualize custom dataset examples (optional)

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob

def read_label(image_path):
  file_name = image_path.replace('.jpg', '.txt')
  with open(file_name, 'rt') as file:
    print(os.path.basename(file_name) + ': \n' + file.read())

image_path = glob.glob("data/ts/*.jpg")
fig = plt.figure(figsize=(12,8))
cols = 2
rows = 2
grid = gridspec.GridSpec(nrows=rows, ncols=cols, figure=fig)
for i in range(cols*rows):
  fig.add_subplot(grid[i])
  image=plt.imread(image_path[i])
  plt.title(os.path.basename(image_path[i]))
  plt.axis(False)
  plt.imshow(image)
  read_label(image_path[i])

plt.savefig("dataset_examples.jpg", dpi=300)

## Modify yolov4 architecture

**Double click on file `yolov4_config.py` to modify the hyperpameters directly from Colab environment**

E.g: I will train my dataset with these parameters:
 - classes= 31, 
 - max_batches=1000
 - batch=64
 - subdivisions=16
 - width=256
 - height=256
 - ...

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# Run python script to create our customize yolov4_custom_train.cfg 
# and yolov4_custom_tes.cfg in folder /cfg
!python yolov4_setup.py

## Create symbolic link in our Drive

Make sure that you laready created directory _YOLOv4_weight/backup_ in your Drive

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# delete backup folder from our 
!rm /content/darknet_for_colab/backup -r

# create Symlinks so we can save trained weight in our Google Drive
# create folder YOLOv4_weight/back in your Drive to store trained weights
!ln -s /content/drive/'My Drive'/YOLOv4_weight/backup /content/darknet_for_colab

In [ ]:
!cp -r /content/darknet_for_colab/cfg /content/drive/'My Drive'/YOLOv4_weight/cfg

## Training

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

!./darknet detector train data/yolov4.data cfg/yolov4_custom_train.cfg yolov4_custom_train_last.weights -dont_show -map
#If you get CUDA out of memory adjust subdivisions above!
#adjust max batches down for shorter training above

## Visualize training result

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# Plotting training result after 2000 epochs
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
train_result = plt.imread("chart.png")
plt.axis(False)
plt.imshow(train_result)

## Test with example image/video

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# download example images and video
!cp /content/drive/MyDrive/examples.zip /content/darknet_for_colab
!unzip examples.zip
!rm -r examples.zip

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# cfg/coco.data was hardcoded in darknet, thus we need 
# to duplicate one with the same content of data/yolov4.data
%cp data/yolov4.data cfg/coco.data

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# test out our detector!
# coco.names is hardcoded somewhere in the detector
img_path = "examples/test_image_4.jpg"
!./darknet detect cfg/yolov4_custom_test.cfg backup/yolov4_custom_train_last.weights {img_path} -dont-show
# !./darknet detect cfg/yolov4_custom_test.cfg backup/yolov4_custom_train_best.weights {img_path} -dont-show

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

import matplotlib.pyplot as plt 
fig = plt.figure(figsize=(12,12))
plt.axis(False)
processed_image = plt.imread("./predictions.jpg")
plt.imshow(processed_image)

In [ ]:
assert os.getcwd()=='/content/darknet_for_colab', 'Directory should be "/content/darknet_for_colab" instead of "{}"'.format(os.getcwd())

# video processed
!python darknet_video.py -v examples/test_video.mp4 -c cfg/yolov4_custom_test.cfg -w backup/yolov4_custom_train_last.weights -o output.mp4

## The End!